# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = 'weather_data.csv'
#filepath = 'weather_data_copy.csv'
weather_data = pd.read_csv(filepath)

weather_data

,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Date Time UTC,Date
0,0,souillac,20.21,64,40,8.23,-20.5167,57.5167,MU,1660243000,2022-08-11
1,1,sao filipe,26.43,72,5,5.16,14.8961,-24.4956,CV,1660243001,2022-08-11
2,2,jamestown,23.67,47,40,5.66,42.0970,-79.2353,US,1660242879,2022-08-11
3,3,kapaa,26.19,76,40,7.72,22.0752,-159.3190,US,1660243004,2022-08-11
4,4,vila velha,23.86,60,0,7.72,-20.3297,-40.2925,BR,1660243005,2022-08-11
...,...,...,...,...,...,...,...,...,...,...,...
549,549,copiapo,12.83,36,100,2.83,-27.3667,-70.3333,CL,1660243810,2022-08-11
550,550,vryheid,14.09,47,0,2.52,-27.7695,30.7917,ZA,1660243811,2022-08-11
551,551,palmer,27.91,46,75,2.06,42.1584,-72.3287,US,1660243813,2022-08-11
552,552,sembe,23.39,82,96,1.61,1.6481,14.5806,CG,1660243816,2022-08-11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps with API key
gmaps.configure(api_key = g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[['Latitude', 'Longitude']]
humidity = weather_data['%Humidity']

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_data


,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Date Time UTC,Date
0,0,souillac,20.21,64,40,8.23,-20.5167,57.5167,MU,1660243000,2022-08-11
1,1,sao filipe,26.43,72,5,5.16,14.8961,-24.4956,CV,1660243001,2022-08-11
2,2,jamestown,23.67,47,40,5.66,42.0970,-79.2353,US,1660242879,2022-08-11
3,3,kapaa,26.19,76,40,7.72,22.0752,-159.3190,US,1660243004,2022-08-11
4,4,vila velha,23.86,60,0,7.72,-20.3297,-40.2925,BR,1660243005,2022-08-11
...,...,...,...,...,...,...,...,...,...,...,...
549,549,copiapo,12.83,36,100,2.83,-27.3667,-70.3333,CL,1660243810,2022-08-11
550,550,vryheid,14.09,47,0,2.52,-27.7695,30.7917,ZA,1660243811,2022-08-11
551,551,palmer,27.91,46,75,2.06,42.1584,-72.3287,US,1660243813,2022-08-11
552,552,sembe,23.39,82,96,1.61,1.6481,14.5806,CG,1660243816,2022-08-11


In [6]:
#finding weather: <50% Hummidity, <10% Cloudiness, temp 4 - 21 C.
ideal_weather = weather_data.loc[weather_data['%Humidity'] < 50]
ideal_weather = ideal_weather.loc[ideal_weather['Cloudiness'] <= 10]
ideal_weather = ideal_weather.loc[(ideal_weather['Temperature'] >= 4) & (ideal_weather['Temperature'] <= 15)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#creating hotel df
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ''

hotel_df.shape

(4, 12)

In [8]:
hotel_list = []
hotel_lat_list = []
hotel_lng_list = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "rankby": "distance",
    "keyword": "hotel",
    "key": g_key,}
       

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]

    params['location'] = f'{lat},{lng}'
    
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel = response['results'][1]['name']
        hotel_lat = response['results'][1]['geometry']['location']['lat']
        hotel_lng = response['results'][1]['geometry']['location']['lng']
                
        hotel_list.append(hotel)
        hotel_lat_list.append(hotel_lat)
        hotel_lng_list.append(hotel_lng)
    
    except:
        print('No hotel found')
        
        hotel = 'no hotel found'
        hotel_list.append(hotel)
    
    

In [9]:
#adding hotel name to hotel list
hotel_df['Hotel Name'] = hotel_list

hotel_df

,Unnamed: 0,City,Temperature,%Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Country,Date Time UTC,Date,Hotel Name
176,176,mvuma,14.47,45,0,2.41,-19.2792,30.5283,ZW,1660243260,2022-08-11,Chivhu Hotel
268,268,uyuni,12.57,10,0,4.60,-20.4597,-66.8250,BO,1660243397,2022-08-11,Yokoso hostal
508,508,yulara,7.71,44,0,4.09,-25.2406,130.9889,AU,1660243749,2022-08-11,Outback Pioneer Hotel & Lodge - Ayers Rock Resort
550,550,vryheid,14.09,47,0,2.52,-27.7695,30.7917,ZA,1660243811,2022-08-11,Oxford Lodge


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
countries = hotel_df["Country"].tolist()
hotel_names = hotel_df["Hotel Name"].tolist()
cities = hotel_df['City'].tolist()

In [12]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, info_box_content= hotel_info)

fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))